# 🔗 TrustChain Interactive Tutorial

**SSL for AI-Agents** — Cryptographic signatures for AI tool responses.

This notebook demonstrates key TrustChain features:
1. Basic tool signing
2. Chain of Trust (linked operations)
3. Multi-tenant isolation
4. Tampering detection
5. Async tools & performance
6. Audit trail visualization

In [4]:
# Install TrustChain if needed
# !pip install -e ..

import asyncio
import time

from trustchain import TrustChain, TrustChainConfig

## 1️⃣ Basic Usage — Signing Tool Responses

Every tool response is automatically signed with Ed25519.

In [5]:
# Create TrustChain instance
tc = TrustChain(TrustChainConfig(enable_nonce=False))

# Define a trusted tool
@tc.tool("weather_api")
def get_weather(city: str):
    """Get current weather for a city."""
    return {
        "city": city,
        "temperature": 22.5,
        "conditions": "Sunny",
        "timestamp": int(time.time())
    }

# Call the tool - response is automatically signed!
response = get_weather("Moscow")

print(f"📍 City: {response.data['city']}")
print(f"🌡️ Temperature: {response.data['temperature']}°C")
print(f"\n🔐 Signature: {response.signature[:48]}...")
print(f"✅ Verified: {response.is_verified}")

📍 City: Moscow
🌡️ Temperature: 22.5°C

🔐 Signature: OV3DIKsR21Iwz53VS4xIgMFWo0HNpeSuAY7Pp4+jiD2pFKWJ...
✅ Verified: True


## 2️⃣ Chain of Trust — Linked Operations

Link operations together for complete audit trails.

In [6]:
tc_chain = TrustChain(TrustChainConfig(enable_nonce=False))

@tc_chain.tool("step_processor")
def process_step(step: int, data: str):
    return {"step": step, "processed": data.upper(), "timestamp": time.time()}

# Build a chain of operations
step1 = tc_chain._signer.sign("step1", {"action": "fetch_data", "source": "API"})
step2 = tc_chain._signer.sign("step2", {"action": "transform", "rows": 1000}, parent_signature=step1.signature)
step3 = tc_chain._signer.sign("step3", {"action": "store", "destination": "DB"}, parent_signature=step2.signature)

print("⛓️ Chain of Trust:")
print(f"  Step 1: {step1.data['action']} → {step1.signature[:24]}...")
print(f"  Step 2: {step2.data['action']} → {step2.signature[:24]}... (parent: {step2.parent_signature[:16]}...)")
print(f"  Step 3: {step3.data['action']} → {step3.signature[:24]}... (parent: {step3.parent_signature[:16]}...)")

# Verify the entire chain
from trustchain.v2.merkle import MerkleTree

tree = MerkleTree.from_chunks([step1.signature, step2.signature, step3.signature])

print(f"\n🌳 Merkle Root: {tree.root[:32]}...")

⛓️ Chain of Trust:
  Step 1: fetch_data → hUddPScmnwulEC2iFIGn4Tkf...
  Step 2: transform → BOfUXP6fk+RG0XS1PekTopjW... (parent: hUddPScmnwulEC2i...)
  Step 3: store → lHhSbak/zcru8DEMVUhG6Xr/... (parent: BOfUXP6fk+RG0XS1...)

🌳 Merkle Root: 061f9ff461ea96d53cd33a3961425cce...


## 3️⃣ Multi-Tenant Isolation

Each tenant has isolated keys — signatures from one tenant cannot verify in another.

In [7]:
from trustchain.v2.tenants import TenantManager

# Create tenant manager
manager = TenantManager()

# Create isolated tenants (get_or_create returns TrustChain instance)
tc_a = manager.get_or_create("company_a")
tc_b = manager.get_or_create("company_b")

print("🏢 Tenant A key ID:", tc_a.get_key_id()[:16] + "...")
print("🏢 Tenant B key ID:", tc_b.get_key_id()[:16] + "...")

# Sign with Tenant A's TrustChain
response_a = tc_a._signer.sign("api_call", {"data": "secret", "tenant": "A"})

# Try to verify with Tenant B (will fail!)
is_valid_b = tc_b.verify(response_a)
print(f"\n❌ Cross-tenant verification: {is_valid_b}")

# Verify with correct tenant
is_valid_a = tc_a.verify(response_a)
print(f"✅ Same-tenant verification: {is_valid_a}")

🏢 Tenant A key ID: 87c3bb67-c4fe-4d...
🏢 Tenant B key ID: 1691ed10-ca13-47...

❌ Cross-tenant verification: False
✅ Same-tenant verification: True


## 4️⃣ Tampering Detection — Security Demo

Any modification to signed data is immediately detected.

In [8]:
tc_secure = TrustChain(TrustChainConfig(enable_nonce=False))

@tc_secure.tool("bank_api")
def get_balance(account_id: str):
    return {"account": account_id, "balance": 1000.00, "currency": "USD"}

# Get legitimate response
response = get_balance("ACC-12345")
print(f"💰 Original balance: ${response.data['balance']}")
print(f"✅ Original verified: {tc_secure.verify(response)}")

# Simulate tampering attack
import copy

tampered = copy.deepcopy(response)
tampered.data["balance"] = 999999.99  # Attacker modifies balance!

print(f"\n🦹 Tampered balance: ${tampered.data['balance']}")
print(f"🚫 Tampered verified: {tc_secure.verify(tampered)}")
print("\n✅ Tampering detected! Signature doesn't match modified data.")

💰 Original balance: $1000.0
✅ Original verified: True

🦹 Tampered balance: $999999.99
🚫 Tampered verified: False

✅ Tampering detected! Signature doesn't match modified data.


## 5️⃣ Async Tools & Performance

TrustChain supports async tools with sub-millisecond signing overhead.

In [9]:
tc_async = TrustChain(TrustChainConfig(enable_nonce=False))

@tc_async.tool("async_processor")
async def process_data(items: list):
    """Async data processing with signature."""
    await asyncio.sleep(0.01)  # Simulate async work
    return {
        "count": len(items),
        "sum": sum(items),
        "avg": sum(items) / len(items) if items else 0
    }

# Measure performance
async def benchmark():
    times = []
    for _ in range(100):
        start = time.perf_counter()
        await process_data([1, 2, 3, 4, 5])
        times.append((time.perf_counter() - start) * 1000)
    return times

times = await benchmark()
avg_time = sum(times) / len(times)

print("⚡ Performance (100 calls):")
print(f"   Average: {avg_time:.2f}ms")
print(f"   Min: {min(times):.2f}ms")
print(f"   Max: {max(times):.2f}ms")
print(f"   Throughput: ~{1000/avg_time:.0f} ops/sec")

⚡ Performance (100 calls):
   Average: 11.02ms
   Min: 10.31ms
   Max: 12.19ms
   Throughput: ~91 ops/sec


## 6️⃣ Audit Trail Visualization

Generate interactive HTML reports of your audit trail.

In [10]:
from trustchain.ui.explorer import ChainExplorer

# Collect responses
tc_audit = TrustChain(TrustChainConfig(enable_nonce=False))

@tc_audit.tool("audit_demo")
def demo_action(action: str):
    return {"action": action, "timestamp": time.time()}

# Generate some actions
responses = [
    demo_action("user_login"),
    demo_action("data_access"),
    demo_action("report_generated"),
    demo_action("user_logout")
]

# Create explorer and export
explorer = ChainExplorer(responses, tc_audit)
html_path = explorer.export_html("audit_report.html")

print(f"📊 Audit report generated: {html_path}")
print(f"   Operations: {len(responses)}")
print(f"   All verified: {all(r.is_verified for r in responses)}")

# Also export as JSON
json_data = explorer.to_json()
print(f"\n📄 JSON export available ({len(json_data)} bytes)")

📊 Audit report generated: audit_report.html
   Operations: 4
   All verified: True

📄 JSON export available (1423 bytes)


## 7️⃣ Real-World Use Case: RAG Pipeline

Secure a Retrieval-Augmented Generation pipeline with cryptographic provenance.

In [11]:
tc_rag = TrustChain(TrustChainConfig(enable_nonce=False))

@tc_rag.tool("document_retriever")
def retrieve_documents(query: str, top_k: int = 3):
    """Retrieve relevant documents for a query."""
    # Simulated document retrieval
    docs = [
        {"id": "doc1", "text": "TrustChain provides cryptographic signatures...", "score": 0.95},
        {"id": "doc2", "text": "AI hallucinations can be prevented with verification...", "score": 0.87},
        {"id": "doc3", "text": "Ed25519 is used for high-performance signing...", "score": 0.82}
    ]
    return {"query": query, "documents": docs[:top_k], "total_found": len(docs)}

@tc_rag.tool("llm_synthesizer")
def synthesize_answer(documents: list, query: str):
    """Synthesize answer from documents."""
    # Simulated LLM response
    return {
        "answer": "TrustChain uses Ed25519 signatures to prevent AI hallucinations.",
        "sources": [d["id"] for d in documents],
        "confidence": 0.92
    }

# Execute RAG pipeline
retrieval = retrieve_documents("How does TrustChain work?")
synthesis = synthesize_answer(retrieval.data["documents"], "How does TrustChain work?")

print("🔍 RAG Pipeline with Cryptographic Provenance:")
print("\n1. Retrieval Step:")
print(f"   Query: '{retrieval.data['query']}'")
print(f"   Documents found: {retrieval.data['total_found']}")
print(f"   Signature: {retrieval.signature[:32]}...")

print("\n2. Synthesis Step:")
print(f"   Answer: '{synthesis.data['answer']}'")
print(f"   Sources: {synthesis.data['sources']}")
print(f"   Confidence: {synthesis.data['confidence']}")
print(f"   Signature: {synthesis.signature[:32]}...")

print("\n✅ Both steps cryptographically verified!")
print(f"   Retrieval valid: {tc_rag.verify(retrieval)}")
print(f"   Synthesis valid: {tc_rag.verify(synthesis)}")

🔍 RAG Pipeline with Cryptographic Provenance:

1. Retrieval Step:
   Query: 'How does TrustChain work?'
   Documents found: 3
   Signature: nhMSHwhbxrdkKeOjZmD8IdWqRiHZKX5A...

2. Synthesis Step:
   Answer: 'TrustChain uses Ed25519 signatures to prevent AI hallucinations.'
   Sources: ['doc1', 'doc2', 'doc3']
   Confidence: 0.92
   Signature: +HdHUVZbuSJ2c3b09F+JNRgsc/rYS79s...

✅ Both steps cryptographically verified!
   Retrieval valid: True
   Synthesis valid: True


## 📚 Summary

TrustChain provides:

| Feature | Description |
|---------|-------------|
| 🔐 Ed25519 Signatures | High-performance cryptographic signing |
| ⛓️ Chain of Trust | Linked operations with Merkle proofs |
| 🏢 Multi-Tenancy | Isolated keys per tenant |
| 🛡️ Tampering Detection | Instant detection of data modification |
| ⚡ Performance | <0.15ms latency, >9000 ops/sec |
| 📊 Audit Trail | HTML visualization and JSON export |

**Learn more:** [GitHub](https://github.com/petro1eum/trust_chain) | [Documentation](https://github.com/petro1eum/trust_chain/wiki)